# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 1:06PM,258381,20,9217681,"Exact-Rx, Inc.",Released
1,Feb 28 2023 1:06PM,258381,20,9217682,"Exact-Rx, Inc.",Released
2,Feb 28 2023 12:44PM,258379,21,WireDecking3,"NBTY Global, Inc.",Executing
3,Feb 28 2023 12:28PM,258377,10,SOTAH0000920,"Nextsource Biotechnology, LLC",Released
4,Feb 28 2023 12:28PM,258377,10,SOTAH0000922,"Nextsource Biotechnology, LLC",Released
5,Feb 28 2023 12:28PM,258377,10,SOTAH0000921,"Nextsource Biotechnology, LLC",Released
6,Feb 28 2023 12:26PM,258376,19,ELI030123,Eli Lilly and Company,Released
7,Feb 28 2023 12:21PM,258373,10,Enova-11425,Emerginnova,Executing
8,Feb 28 2023 12:21PM,258373,10,DNMD-35297,Emerginnova,Released
9,Feb 28 2023 12:21PM,258373,10,Enova-11426,Emerginnova,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258373,Released,2
33,258376,Released,1
34,258377,Released,3
35,258379,Executing,1
36,258381,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258373,NaN,2.0,2.0
258376,NaN,NaN,1.0
258377,NaN,NaN,3.0
258379,NaN,1.0,NaN
258381,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,1.0,6.0,0.0
258251,1.0,18.0,1.0
258260,3.0,14.0,4.0
258296,0.0,0.0,57.0
258307,0.0,0.0,38.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,1,6,0
258251,1,18,1
258260,3,14,4
258296,0,0,57
258307,0,0,38


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,1,6,0
1,258251,1,18,1
2,258260,3,14,4
3,258296,0,0,57
4,258307,0,0,38


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,1,6,
1,258251,1,18,1
2,258260,3,14,4
3,258296,,,57
4,258307,,,38


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc."
2,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc."
3,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC"
6,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company
7,Feb 28 2023 12:21PM,258373,10,Emerginnova
11,Feb 28 2023 11:55AM,258371,10,Bio-PRF
12,Feb 28 2023 11:52AM,258370,19,ACG North America LLC
13,Feb 28 2023 11:26AM,258368,10,Eywa Pharma Inc.
26,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc."
27,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",,,2
1,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",,1,
2,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",,,3
3,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,,,1
4,Feb 28 2023 12:21PM,258373,10,Emerginnova,,2,2
5,Feb 28 2023 11:55AM,258371,10,Bio-PRF,,,1
6,Feb 28 2023 11:52AM,258370,19,ACG North America LLC,,,1
7,Feb 28 2023 11:26AM,258368,10,Eywa Pharma Inc.,,,13
8,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",,,1
9,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2,,
1,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",,1,
2,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3,,
3,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1,,
4,Feb 28 2023 12:21PM,258373,10,Emerginnova,2,2,
5,Feb 28 2023 11:55AM,258371,10,Bio-PRF,1,,
6,Feb 28 2023 11:52AM,258370,19,ACG North America LLC,1,,
7,Feb 28 2023 11:26AM,258368,10,Eywa Pharma Inc.,13,,
8,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",1,,
9,Feb 28 2023 11:21AM,258366,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2,,
1,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",,1,
2,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3,,
3,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1,,
4,Feb 28 2023 12:21PM,258373,10,Emerginnova,2,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2.0,NaN,NaN
1,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",NaN,1.0,NaN
2,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
3,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1.0,NaN,NaN
4,Feb 28 2023 12:21PM,258373,10,Emerginnova,2.0,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2.0,0.0,0.0
1,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",0.0,1.0,0.0
2,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
3,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1.0,0.0,0.0
4,Feb 28 2023 12:21PM,258373,10,Emerginnova,2.0,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4908631,257.0,3.0,0.0
12,258342,0.0,1.0,0.0
15,774736,5.0,38.0,5.0
19,775076,2.0,1.0,0.0
20,258381,2.0,0.0,0.0
21,258379,0.0,1.0,0.0
22,775098,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4908631,257.0,3.0,0.0
1,12,258342,0.0,1.0,0.0
2,15,774736,5.0,38.0,5.0
3,19,775076,2.0,1.0,0.0
4,20,258381,2.0,0.0,0.0
5,21,258379,0.0,1.0,0.0
6,22,775098,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,257.0,3.0,0.0
1,12,0.0,1.0,0.0
2,15,5.0,38.0,5.0
3,19,2.0,1.0,0.0
4,20,2.0,0.0,0.0
5,21,0.0,1.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,257.0
1,12,Released,0.0
2,15,Released,5.0
3,19,Released,2.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,5.0,0.0,0.0,0.0,0.0
Executing,3.0,1.0,38.0,1.0,0.0,1.0,0.0
Released,257.0,0.0,5.0,2.0,2.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,5.0,0.0,0.0,0.0,0.0
1,Executing,3.0,1.0,38.0,1.0,0.0,1.0,0.0
2,Released,257.0,0.0,5.0,2.0,2.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,5.0,0.0,0.0,0.0,0.0
1,Executing,3.0,1.0,38.0,1.0,0.0,1.0,0.0
2,Released,257.0,0.0,5.0,2.0,2.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()